In [64]:
import random
import json
import pickle
import numpy as np
import nltk

nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers.legacy import SGD
from tensorflow.keras.models import load_model

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\angel\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


#### Training the Chatbot

In [2]:
lemmatizer = WordNetLemmatizer()

intents = json.loads(open("Intents.json").read())

In [3]:

words = []
classes = []
documents = []
ignore_letters = ["?",",","!",".","'"]

for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        word_list = word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,[intent["tag"]]))

        if intent["tag"] not in classes:
            classes.append(intent["tag"])

print(documents)



[(['hello'], ['Greetings']), (['hey'], ['Greetings']), (['hi'], ['Greetings']), (['how', 'are', 'you', '?'], ['Greetings']), (['what', "'s", 'up'], ['Greetings']), (['how', 'is', 'it', 'going'], ['Greetings']), (['greetings'], ['Greetings']), (['what', 'is', 'your', 'name'], ['name']), (['and', 'you', 'are', '?'], ['name']), (['what', 'should', 'I', 'call', 'you'], ['name']), (['who', 'are', 'you'], ['name']), (['what', "'s", 'your', 'name'], ['name']), (['what', 'is', 'pcos'], ['PCOS']), (['tell', 'me', 'about', 'pcos'], ['PCOS']), (['what', 'is', 'poly', 'cystic', 'ovarian', 'syndrome'], ['PCOS'])]


In [4]:
words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(classes)

In [5]:
pickle.dump(words, open("words.pickle","wb"))
pickle.dump(classes, open("classes.pickle","wb"))

In [7]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)

    output_row[classes.index(document[1][0])] = 1

    training.append([bag, output_row]) 

random.shuffle(training)
training = np.array(training)



C:\Users\angel\AppData\Local\Temp\ipykernel_6968\2317465093.py:18: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [29]:
# features and labels to train the NR
train_x = list(training[:,0])
train_y = list(training[:,1])

In [50]:
len(train_x[0])

28

In [62]:
#building the model
#create optimizer

model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]),), activation = "relu")) # input_shape should be a tuple
model.add(Dropout(0.5))
model.add(Dense(64, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = "softmax"))

sgd = SGD(learning_rate = 0.01, decay = 0.000001,momentum=0.9,nesterov=True)

model.compile(loss = "categorical_crossentropy",optimizer = "sgd", metrics = ["accuracy"])


In [63]:
#fit model
model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size = 5, verbose = 1)
model.save("chatbot_model.model")
print("Done")


Epoch 1/200
3/3 [==============================] - 8s 68ms/step - loss: 1.0903 - accuracy: 0.3333
Epoch 2/200
3/3 [==============================] - 0s 10ms/step - loss: 1.0603 - accuracy: 0.3333
Epoch 3/200
3/3 [==============================] - 0s 9ms/step - loss: 1.1667 - accuracy: 0.3333
Epoch 4/200
3/3 [==============================] - 0s 10ms/step - loss: 1.1677 - accuracy: 0.3333
Epoch 5/200
3/3 [==============================] - 0s 10ms/step - loss: 1.0823 - accuracy: 0.4667
Epoch 6/200
3/3 [==============================] - 0s 8ms/step - loss: 1.1168 - accuracy: 0.0667
Epoch 7/200
3/3 [==============================] - 0s 8ms/step - loss: 1.0885 - accuracy: 0.4667
Epoch 8/200
3/3 [==============================] - 0s 14ms/step - loss: 1.0255 - accuracy: 0.4667
Epoch 9/200
3/3 [==============================] - 0s 8ms/step - loss: 1.0444 - accuracy: 0.4000
Epoch 10/200
3/3 [==============================] - 0s 9ms/step - loss: 1.0302 - accuracy: 0.6000
Epoch 11/200
3/3 [======

INFO:tensorflow:Assets written to: chatbot_model.model\assets


INFO:tensorflow:Assets written to: chatbot_model.model\assets


Done


### Test Chatbot